# Insertion Agentic Workflow

In [2]:
from typing import TypedDict, List
from langchain_core.messages import AnyMessage
from typing import TypedDict, List, Annotated
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from langchain_core.messages import ToolMessage
from langgraph.prebuilt import create_react_agent
# --- State ---
from typing import TypedDict, List, Annotated
from langchain_core.messages import AnyMessage
import operator




# --- State ---
class AgentState(TypedDict):
    user_input: str
    # Use add_messages reducer for messages to handle concurrent appends
    messages: Annotated[List[AnyMessage], add_messages]
        
    # These are single values, so they're fine as-is
    has_context: bool
    final_answer: str
    # Add this to track the insertion agent message
    insertion_agent_message: AnyMessage

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_output_tokens=1000  # limiting the output affects the quality of tool calling
)

# Insertion

In [4]:
import os
import re
from typing import List, Dict, Optional
from dotenv import load_dotenv
import psycopg2
from psycopg2.extras import RealDictCursor
from sentence_transformers import SentenceTransformer
from huggingface_hub import login
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
import logging


load_dotenv()

login(token=os.getenv("HUGGINGFACE_TOKEN"))
CONNECTION_STRING = os.getenv("DATABASE_URL")
SECRET_KEY = os.getenv("DATABASE_ENCRYPTION_KEY")

engine = create_engine(
    CONNECTION_STRING,
    pool_size=5,
    max_overflow=10,
    pool_pre_ping=True,
    pool_recycle=3600,
    connect_args={
        "keepalives": 1,
        "keepalives_idle": 30,
        "keepalives_interval": 10,
        "tcp_user_timeout": 60000,
    },
    echo=False
)

# Load model once globally
model = SentenceTransformer("google/embeddinggemma-300m")

In [5]:
def embed(text: str) -> list:
    """Generate embedding vector from text"""
    if not text or not isinstance(text, str):
        raise ValueError("Query must be a non-empty string")
    return model.encode(text, normalize_embeddings=True).tolist()  # ✅ Normalize for cosine similarity

# Setting Up Default Profile

In [6]:
ELDERLY_ID = "12345678-1234-1234-1234-012345678910"

# Elderly profile data
profile_data = {
    "name": "Admiralty Bedok Canberra Tan",
    "date_of_birth": "1965-01-01",
    "gender": "Male",  
    "nationality": "Singaporean",
    "dialect_group": "Hokkien",
    "marital_status": "Married",
    "address": "38 Oxley Road, Singapore 238629"
}

try:
    with psycopg2.connect(CONNECTION_STRING) as conn:
        with conn.cursor() as cur:
            print("Creating elderly profile...")
            
            # Insert profile with fixed UUID
            cur.execute("""
                INSERT INTO elderly_profile 
                (id, name, date_of_birth, gender, nationality, dialect_group, marital_status, address)
                VALUES (%s, pgp_sym_encrypt(%s,%s), pgp_sym_encrypt(%s,%s), %s, 
                        pgp_sym_encrypt(%s,%s), pgp_sym_encrypt(%s,%s), %s, pgp_sym_encrypt(%s,%s))
                ON CONFLICT (id) DO NOTHING;
            """, (
                ELDERLY_ID,
                profile_data["name"], SECRET_KEY,
                profile_data["date_of_birth"], SECRET_KEY,
                profile_data["gender"],
                profile_data["nationality"], SECRET_KEY,
                profile_data["dialect_group"], SECRET_KEY,
                profile_data["marital_status"],
                profile_data["address"], SECRET_KEY
            ))
            
            print(f"Profile created/verified with ID: {ELDERLY_ID}")
            
except Exception as e:
    print(f"Error: {e}")

Creating elderly profile...
Profile created/verified with ID: 12345678-1234-1234-1234-012345678910


# Insertion Functions

### Functions

In [7]:
def insert_short_term(content: str, elderly_id: str = ELDERLY_ID) -> dict:
    if not content or not content.strip():
        return {"success": False, "error": "content is required and cannot be empty."}

    embedding = embed(content)

    try:
        with engine.connect() as conn:
            query = text("""
                INSERT INTO short_term_memory (
                    elderly_id, content, embedding
                ) VALUES (
                    :elderly_id, :content, :embedding
                )
                RETURNING id, created_at;
            """)
            result = conn.execute(query, {
                "elderly_id": elderly_id.strip(),
                "content": content.strip(),
                "embedding": str(embedding)
            }).fetchone()
            conn.commit()

            return {
                "success": True,
                "message": "Short-term memory stored successfully",
                "record_id": str(result.id),
                "created_at": result.created_at.isoformat() if result.created_at else None,
                "embedding_provided": embedding is not None
            }

    except SQLAlchemyError as e:
        logging.error(f"❌ Database error inserting STM: {str(e)}")
        return {"success": False, "error": f"Database error: {str(e)}"}
    except Exception as e:
        logging.error(f"Unexpected error inserting STM: {str(e)}")
        return {"success": False, "error": f"Unexpected error: {str(e)}"}



def insert_long_term(category: str, key: str, value: str, elderly_id: str = ELDERLY_ID) -> dict:
    if not category or not category.strip():
        return {"success": False, "error": "category is required and cannot be empty."}
    if not key or not key.strip():
        return {"success": False, "error": "key is required and cannot be empty."}
    if not value or not value.strip():
        return {"success": False, "error": "value is required and cannot be empty."}

    embedding = embed(value)

    try:
        with engine.connect() as conn:
            query = text("""
                INSERT INTO long_term_memory (
                    elderly_id, category, key, value, embedding
                ) VALUES (
                    :elderly_id, :category, :key, :value, :embedding
                )
                RETURNING id, last_updated;
            """)
            result = conn.execute(query, {
                "elderly_id": elderly_id.strip(),
                "category": category.strip(),
                "key": key.strip(),
                "value": value.strip(),
                "embedding": str(embedding)
            }).fetchone()
            conn.commit()

            return {
                "success": True,
                "message": "Long-term memory stored successfully",
                "record_id": str(result.id),
                "last_updated": result.last_updated.isoformat() if result.last_updated else None,
                "embedding_provided": embedding is not None
            }

    except SQLAlchemyError as e:
        logging.error(f"❌ Database error inserting LTM: {str(e)}")
        return {"success": False, "error": f"Database error: {str(e)}"}
    except Exception as e:
        logging.error(f"Unexpected error inserting LTM: {str(e)}")
        return {"success": False, "error": f"Unexpected error: {str(e)}"}



def insert_health_record(record_type: str, description: str, diagnosis_date: Optional[str] = None, elderly_id: str = ELDERLY_ID) -> dict:
    if not record_type or not record_type.strip():
        return {"success": False, "error": "record_type is required and cannot be empty."}
    if not description or not description.strip():
        return {"success": False, "error": "description is required and cannot be empty."}

    # Validate date format if provided
    if diagnosis_date:
        try:
            from datetime import datetime
            datetime.strptime(diagnosis_date, "%Y-%m-%d")
        except ValueError:
            return {"success": False, "error": "diagnosis_date must be in YYYY-MM-DD format if provided."}

    try:
        with engine.connect() as conn:
            query = text("""
                INSERT INTO healthcare_records (
                    elderly_id, record_type, description, diagnosis_date, embedding
                ) VALUES (
                    :elderly_id, :record_type, :description, :diagnosis_date, :embedding
                )
                RETURNING id, last_updated;
            """)
            result = conn.execute(query, {
                "elderly_id": elderly_id.strip(),
                "record_type": record_type.strip(),
                "description": description.strip(),
                "diagnosis_date": diagnosis_date if diagnosis_date else None,
                "embedding": str(embedding) if embedding else None
            }).fetchone()
            conn.commit()

            return {
                "success": True,
                "message": "Healthcare record stored successfully",
                "record_id": str(result.id),
                "last_updated": result.last_updated.isoformat() if result.last_updated else None,
                "embedding_provided": embedding is not None,
                "diagnosis_date": diagnosis_date
            }

    except SQLAlchemyError as e:
        logging.error(f"❌ Database error inserting health record: {str(e)}")
        return {"success": False, "error": f"Database error: {str(e)}"}
    except Exception as e:
        logging.error(f"Unexpected error inserting health record: {str(e)}")
        return {"success": False, "error": f"Unexpected error: {str(e)}"}

### Tools

#### pydantic for function calling schema for insertion

In [8]:
from typing import Optional
from enum import Enum
from pydantic import BaseModel, Field


class LTMCategories(str, Enum):
    personal = "personal"
    family = "family"
    education = "education"
    career = "career"
    lifestyle = "lifestyle"
    finance = "finance"
    legal = "legal"


class HealthRecordTypes(str, Enum):
    condition = "condition"
    procedure = "procedure"
    appointment = "appointment"
    medication = "medication"


class InsertShortTermSchema(BaseModel):
    content: str = Field(
        ..., 
        description="Short-term conversational detail to store. Use for temporary information that's useful in the near future but doesn't belong in long-term or healthcare storage. Examples: reminders, temporary preferences, upcoming appointments, casual mentions."
    )


class InsertLongTermSchema(BaseModel):
    category: LTMCategories = Field(
        ..., 
        description="Category of long-term memory. Use for stable traits & preferences that rarely change - generally fixed profile information. Must follow one of the following categories ['personal','family','education','career','lifestyle','finance','legal']"
    )
    key: str = Field(
        ..., 
        description="Key or label for the memory fact. Should be a clear, descriptive subcategory for the category for this piece of long-term information"
    )
    value: str = Field(
        ..., 
        description="The fact/value to store. The actual free form user information long-term memory item. This should be stable information that rarely changes."
    )


class InsertHealthSchema(BaseModel):
    record_type: HealthRecordTypes = Field(
        ..., 
        description="Type of healthcare record. Use for official physical/mental health information explicitly shared for future care. Must follow one of the following categories ['condition','procedure','appointment','medication']"
    )
    description: str = Field(
        ..., 
        description="The details of the healthcare record. The actual free form and complete description of the medical information being stored. Should include specific details relevant to the record type."
    )
    diagnosis_date: Optional[str] = Field(
        None, 
        description="Date in YYYY-MM-DD format (optional). When the healthcare event occurred, was diagnosed, or is scheduled. Leave empty if no specific date was mentioned. Examples: '2023-12-15', '2024-03-20', null."
    )

#### @tool to wrap over core insertion functions

In [9]:
from langchain_core.tools import tool


@tool(args_schema=InsertShortTermSchema)
def insert_short_term_tool(content: str) -> str:
    """Insert a short-term memory item."""
    result = insert_short_term(content=content)
    return str(result)


@tool(args_schema=InsertLongTermSchema)
def insert_long_term_tool(category: LTMCategories, key: str, value: str) -> str:
    """Insert a long-term memory fact (stable traits, demographics, preferences)."""
    result = insert_long_term(category=category, key=key, value=value)
    return str(result)


@tool(args_schema=InsertHealthSchema)
def insert_health_tool(record_type: HealthRecordTypes, description: str, diagnosis_date: Optional[str] = None) -> str:
    """Insert a healthcare record (conditions, medications, appointments)."""
    result = insert_health_record(record_type=record_type, description=description, diagnosis_date=diagnosis_date)
    return str(result)

insertion_tools = [insert_long_term_tool, insert_health_tool, insert_short_term_tool]
insertion_llm = llm.bind_tools(insertion_tools)

# Insertion Agent

In [10]:
# ------------- 1.1  System prompt for the storage agent -------------
INSERTION_SYSTEM = """
    ## Role  
    You are memory agent involved in deciding what information is important to store and in the right place. Only store what is neccessary.
    If you decide to store information, Long-Term and Healthcare are for official and formal information, general miscellaneous and all other information should be stored in short term
    
    --------------------------------------------------
    OBJECTIVES  
    1. Extract ONLY relevant user information from the conversation.  
    2. Decide if any item is worth storing.  
    3. If YES → store all relevant information by calling the matching tool(s).  
    4. If NO → do nothing (silent pass).

    --------------------------------------------------
    BUCKETS 

    1. LONG-TERM (ltm)  
    Content: stable traits & preferences that rarely change, generally fixed profile information
    Examples:  
    - name, preferred_name, date_of_birth, gender, address  
    - food/activity/music/hobby preferences (category + value)  
    - family & social relationships (contact_name, relationship_type, is_emergency_contact)  
    - life memories (memory_title, memory_content, memory_category)  
    - daily routines (routine_name, time_of_day, frequency)

    2. HEALTH-CARE (hcm)  
    Tables: Official medical_records, medications, medical_conditions, allergies, diagnosis  
    Content: physical/mental health info explicitly shared for future care
    Examples:  
    - diagnoses, lab results, vital signs (medical_records)  
    - medication_name, dosage, frequency (medications)  
    - condition_name, severity, status (medical_conditions)  
    - allergen, reaction_type (allergies)
    - official tied medical facility (polyclinics, hospitals, family clinics, general practioners)

    3. GENERAL / SHORT-TERM  
    Tables: memory_contexts (context_summary)  
    Content: conversational details useful in the near future
    Examples:  
    - “I have a cardiology visit next Tuesday”  
    - “Please remind me to call my grandson tonight”  
    - “I prefer chicken for dinner today”

    --------------------------------------------------
    RULES  
    - You may call multiple tools in one go if needed.
    - Only store what’s explicitly shared and matches a bucket.
"""

# ------------- 1.2  ReAct agent (no custom state_schema) -------------
react_insertion_agent = create_react_agent(
    model=llm,
    tools=insertion_tools
)

def react_insertion_node(state: AgentState):
    system = SystemMessage(content=INSERTION_SYSTEM)
    input_msg = HumanMessage(content=state["user_input"])
    react_result = react_insertion_agent.invoke({"messages": [system, input_msg]})

    last_ai = next(m for m in reversed(react_result["messages"]) if isinstance(m, AIMessage))

    return {
        "messages": react_result["messages"],
        "insertion_actions": [],
        "insertion_agent_message": last_ai
    }

# Agentic Flow

In [11]:
# conditional path
def route_insertion(state: AgentState):
    # Get the last message (should be from Insertion Agent)
    messages = state.get("messages", [])
    if messages:
        last_message = messages[-1]  # Get the Insertion Agent's response
        if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
            print(f"[TOOL CALL] Routing to execute_insertion, tool_calls: {len(last_message.tool_calls)}")
            return "execute_insertion"
    
    print("[END] Routing to end")
    return "end"

In [12]:
# --- Tool Nodes ---
insertion_tool_node = ToolNode(insertion_tools)

# --- Graph ---
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("Insertion_Agent", react_insertion_node)
workflow.add_node("execute_insertion", insertion_tool_node)

# --- Edges ---
workflow.add_edge(START, "Insertion_Agent")

# Conditional routing: either execute insertion or go straight to END
workflow.add_conditional_edges(
    "Insertion_Agent",
    route_insertion,
    {
        "execute_insertion": "execute_insertion",
        "end": END,
    }
)

# After insertion, go to END
workflow.add_edge("execute_insertion", END)

# --- Compile ---
graph = workflow.compile()

In [13]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [14]:
def create_initial_state(user_input: str) -> AgentState:
    return {
        "user_input": user_input,
        "messages": [],  # 👈 explicitly start empty
        "final_answer": "",
        "insertion_actions": [],  # Initialize as empty list
        "insertion_agent_message": None,
        "has_context": False
    }

# Testing the DAG

In [15]:
input_text = "I stay in 38 Oxley Road, Singapore 238629"
result = graph.invoke(create_initial_state(input_text))

[END] Routing to end


In [16]:
x = result.get("messages")[2]

In [17]:
import json

print(json.dumps(x.dict(), indent=2))


{
  "content": "",
  "additional_kwargs": {
    "function_call": {
      "name": "insert_long_term_tool",
      "arguments": "{\"category\": \"personal\", \"value\": \"38 Oxley Road, Singapore 238629\", \"key\": \"address\"}"
    }
  },
  "response_metadata": {
    "prompt_feedback": {
      "block_reason": 0,
      "safety_ratings": []
    },
    "finish_reason": "STOP",
    "model_name": "gemini-2.5-flash",
    "safety_ratings": []
  },
  "type": "ai",
  "name": null,
  "id": "run--47ea0212-1da1-411b-9045-90149ef20dfc-0",
  "example": false,
  "tool_calls": [
    {
      "name": "insert_long_term_tool",
      "args": {
        "category": "personal",
        "value": "38 Oxley Road, Singapore 238629",
        "key": "address"
      },
      "id": "108ffb63-d733-41c7-8aca-6278607551d1",
      "type": "tool_call"
    }
  ],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 964,
    "output_tokens": 88,
    "total_tokens": 1052,
    "input_token_details": {
      "ca

C:\Users\leeee\AppData\Local\Temp\ipykernel_39840\773403086.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(json.dumps(x.dict(), indent=2))
